# Visualisierung der Druckorte der Düsseldorfer Kreuzherrenbibliothek

In [ ]:
!pip install folium

In [ ]:
import requests
from bs4 import BeautifulSoup as soup
import unicodedata
from lxml import etree
import pandas as pd
from geopy.geocoders import Nominatim
import folium

In [ ]:
def query_chunk(query, index):
  base_url = "https://hbz-ulbd.alma.exlibrisgroup.com/view/sru/49HBZ_DUE"
  params = {'recordSchema' : 'marcxml',
          'operation': 'searchRetrieve',
          'version': '1.2',
          'maximumRecords': '50',
          'query': query,
          'startRecord': index
         }
  r = requests.get(base_url, params=params)
  xml = soup(r.content, features="xml")
  record_data = xml.find_all('recordData')
  records = []
  for record in record_data:
    records.append(record.find('record'))
  if len(records) == 50:
    records += query_chunk(query, index+50)
  return records

In [ ]:
def extract_field(record, datafield, subfield):
  # s. https://github.com/deutsche-nationalbibliothek/dnblab/blob/main/DNB_SRU_Tutorial.ipynb
  ns = {"marc":"http://www.loc.gov/MARC21/slim"}
  # Die DNB wird schon irgendeinen Grund für die Unicode-Normalisierung haben...
  xml = etree.fromstring(unicodedata.normalize("NFC", str(record)))
  result = xml.xpath(f"marc:datafield[@tag = '{datafield}']/marc:subfield[@code = '{subfield}']", namespaces=ns)
  try:
    result = result[0].text
  except:
    result = None
  return result

In [ ]:
records = query_chunk('local_field_980=KHB*', 0)
places  = []
for record in records:
  place = extract_field(record, '264', 'a')
  if place:
    place = place.translate(str.maketrans('', '', '[]?()'))
    place = place.replace('æ', 'ae')
    place = place.replace('ae', 'e')
    place = place.strip()
    places.append(place)
print(places)

df = pd.DataFrame(places, columns=['places'])
df['places'].value_counts()


['Nürnberg', 'Paris', 'Köln', 'Lugduni', 'Lipsi', 'Parhisiis', 'Köln', 'Rome', 'Köln', 'Francofvrti', 'Geneve', 'Venedig', 'Colonie', 'Niederlande', 'Köln', 'Köln', 'Hagenaw', 'Lugdunensium', 'Colonie', 'Speyer', 'Nürnberg', 'Alost', 'Avignon', 'Colonie Agrippine', 'Colonie', 'Köln', 'Basel', 'Lovanii', 'Antwerpen', 'Antuerpie', 'Parisiis', 'Köln', 'Köln', 'Köln', 'Köln', 'Basilee', 'Colonie', 'Lovanii', 'Straßburg', 'Straßburg', 'Basel', 'Speyer', 'Parisiis', 'Köln', 'Köln', 'Köln', 'Colonie', 'Colonie', 'Frankfurt, Main', 'Lauingen', 'Köln', 'Basel', 'Köln', 'Mainz', 'Köln', 'Köln', 'Köln', 'Niederlande', 'Köln', 'Straßburg', 'Basel', 'Löwen', 'Basilee', 'Köln', 'Köln', 'Köln', 'Köln', 'Mainz', 'Köln', 'Strassburg', 'Basel', 'Augsburg', 'Colonie', 'Köln', 'Parrhisiensi', 'Parrhisiensi', 'Parisiis', 'Colonie', 'Köln', 'Colonie', 'Colonia', 'Agrippine', 'Basel', 'Basel', 'Augsburg', 'Deventer', 'Köln', 'Köln', 'Deventer', 'Basel', 'Basel', 'Colonie', 'Köln', 'Köln', 'Apvd Inclytam Germ

,count
places,
"Düsseldorf, Kreuzherrenkonvent",74
Köln,72
Colonie,19
Basel,18
Basilee,11
...,...
Lyptzigk,1
Ingolstadii,1
Magdebvrge,1


In [ ]:
replacements = {
    "Colonie": "Köln",
    "Colonia": "Köln",
    "Colonie Agrippine": "Köln",
    "Agrippine": "Köln",
    "Impressum Colonie": "Köln",
    "Köln oder Düsseldorf, Kreuzherrenkonvent": "Köln",
    "Basilee": "Basel",
    "Basilie": "Basel",
    "Basiliee": "Basel",
    "Basileam": "Basel",
    "Apvd Inclytam Germanie Basileam": "Basel",
    "Strassburg": "Straßburg",
    "Argentine": "Straßburg",
    "Argentorati": "Straßburg",
    "Strassburg und Basel": "Straßburg",
    "Parisiis": "Paris",
    "Parrhisiensi": "Paris",
    "Parrhisiis": "Paris",
    "Parrisijs": "Paris",
    "Parrhysii": "Paris",
    "Parissus": "Paris",
    "Parhisiis": "Paris",
    "Parhisijs": "Paris",
    "Parrhisiensis": "Paris",
    "Hagenaw": "Hagenau",
    "Haganoe": "Hagenau",
    "Venetiis": "Venedig",
    "Uenetijs": "Venedig",
    "Venetia": "Venedig",
    "Antuerpie": "Antwerpen",
    "Antverpie": "Antwerpen",
    "Lovanii": "Löwen",
    "Lugduni": "Lyon",
    "Lvgdvni": "Lyon",
    "Lugdunensium": "Lyon",
    "Lugdunensis": "Lyon",
    "Francofvrti": "Frankfurt, Main",
    "Rome": "Rom",
    "Lipsi": "Leipzig",
    "Lyptzigk": "Leipzig",
    "Geneve": "Genf",
    "Magdebvrge": "Magdeburg",
    "Ingolstadii": "Ingolstadt",
    "Monasterij Vestphalie": "Münster",
    "Düsseldorf, Kreuzherrenkonvent": "Düsseldorf",
    "nördl. Rheinland Düsseldorf": "Düsseldorf",
    "Düsseldorf, Kreuzherrenkonvent ; Niederrhein; Deventer, Fraterherrenkonvent": "Düsseldorf",
    "Düsseldorf oder Köln, Kreuzherrenkonvent": "Düsseldorf",
    "Maaseik / Düsseldorf, Kreuzherrenkonvent": "Maaseik",
    "S.l.": "",
    "Cen": ""
}
for old_name, new_name in replacements.items():
    df = df.replace(old_name, new_name)

counts = df['places'].value_counts()
counts_df = counts.rename_axis('places').reset_index(name='counts')
display(counts_df)

,places,counts
0,Köln,97
1,Düsseldorf,79
2,Basel,33
3,Straßburg,24
4,Paris,19
5,Venedig,15
6,Nürnberg,8
7,Hagenau,7
8,Antwerpen,6
9,Lyon,5


In [ ]:
geolocator = Nominatim(user_agent="py-ulb")

def get_coordinates(place_name):
  try:
    location = geolocator.geocode(place_name, timeout=None)
    if location:
      return (location.latitude, location.longitude)
  except:
    return None

counts_df['coordinates'] = counts_df['places'].apply(get_coordinates)
display(counts_df)


,places,counts,coordinates
0,Köln,97,"(50.938361, 6.959974)"
1,Düsseldorf,79,"(51.2254018, 6.7763137)"
2,Basel,33,"(47.5581077, 7.5878261)"
3,Straßburg,24,"(48.584614, 7.7507127)"
4,Paris,19,"(48.8534951, 2.3483915)"
5,Venedig,15,"(45.4371908, 12.3345898)"
6,Nürnberg,8,"(49.453872, 11.077298)"
7,Hagenau,7,"(48.8172236, 7.7885978)"
8,Antwerpen,6,"(51.2211097, 4.3997081)"
9,Lyon,5,"(45.7578137, 4.8320114)"


In [ ]:
m = folium.Map(location=[counts_df['coordinates'].str[0].mean(), counts_df['coordinates'].str[1].mean()], zoom_start=5)

for index, row in counts_df.iterrows():
  if row['coordinates']:
    folium.Marker([row['coordinates'][0], row['coordinates'][1]], popup=f"{row['places']}: {row['counts']}").add_to(m)

m